# 02. Data Layer Deep Dive

Iceberg 테이블의 **Data Layer**를 구성하는 Parquet 파일의 내부 구조를 직접 분석합니다.

## Data Layer 개요

Iceberg의 Data Layer는 실제 레코드가 저장된 **데이터 파일**들로 구성됩니다.

- 기본 포맷: **Apache Parquet** (ORC, Avro도 지원)
- Parquet는 **Columnar Format** (열 기반 저장)
  - 분석 쿼리에서 필요한 컬럼만 읽어 I/O를 줄임
  - 같은 컬럼의 데이터를 모아 저장하므로 **압축 효율**이 높음

### Parquet 파일 내부 구조

```
┌──────────────────────────┐
│       Row Group 1        │
│  ┌────┬────┬────┬────┐  │
│  │Col1│Col2│Col3│... │  │  ← Column Chunks
│  └────┴────┴────┴────┘  │
├──────────────────────────┤
│       Row Group 2        │
│  ┌────┬────┬────┬────┐  │
│  │Col1│Col2│Col3│... │  │
│  └────┴────┴────┴────┘  │
├──────────────────────────┤
│        Footer            │
│  (Schema, Row Group      │
│   metadata, statistics)  │
└──────────────────────────┘
```

- **Row Group**: 행의 묶음 (기본 128MB)
- **Column Chunk**: Row Group 내 단일 컬럼의 데이터
- **Footer**: 스키마, 통계 등 메타데이터

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size

In [ ]:
spark = create_spark_session("DataLayerDeepDive")

## 테이블 생성 및 데이터 삽입

200건의 데이터를 삽입하여 Data Layer를 구성합니다.

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.data_orders")

spark.sql("""
    CREATE TABLE demo.lab.data_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

orders = generate_orders(num_records=200, seed=42)
df = to_spark_df(spark, orders)
df.writeTo("demo.lab.data_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.data_orders").collect()[0]["cnt"]
print(f"삽입 완료: {count}건")

## 디렉토리 구조 확인

In [ ]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/data_orders"

show_tree(TABLE_PATH)

## Parquet 파일 찾기

`glob`을 사용하여 data/ 디렉토리 내 모든 Parquet 파일을 찾습니다.

In [ ]:
import glob
import os

parquet_files = sorted(glob.glob(f"{TABLE_PATH}/data/**/*.parquet", recursive=True))

print(f"Parquet 파일 수: {len(parquet_files)}개\n")
for f in parquet_files:
    rel = os.path.relpath(f, TABLE_PATH)
    size_kb = os.path.getsize(f) / 1024
    print(f"  {rel}  ({size_kb:.1f} KB)")

## 파티션 디렉토리 구조 관찰

Iceberg의 Hidden Partitioning에 의해 `order_date_month=YYYY-MM` 형태의 디렉토리가 생성됩니다.

In [ ]:
data_dir = os.path.join(TABLE_PATH, "data")
partitions = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

print(f"파티션 수: {len(partitions)}개\n")
for p in partitions:
    p_path = os.path.join(data_dir, p)
    files = [f for f in os.listdir(p_path) if f.endswith(".parquet")]
    p_size = sum(os.path.getsize(os.path.join(p_path, f)) for f in files)
    print(f"  {p}: {len(files)}개 파일, {p_size / 1024:.1f} KB")

## Parquet 파일 내부 분석 (pyarrow)

`pyarrow.parquet`를 사용하여 Parquet 파일의 내부 구조를 분석합니다.

In [ ]:
import pyarrow.parquet as pq

# 첫 번째 Parquet 파일 분석
sample_file = parquet_files[0]
print(f"분석 파일: {os.path.relpath(sample_file, TABLE_PATH)}")
print("=" * 60)

pf = pq.ParquetFile(sample_file)
meta = pf.metadata

print(f"\n[ 기본 정보 ]")
print(f"  Parquet 포맷 버전: {meta.format_version}")
print(f"  생성 라이브러리:   {meta.created_by}")
print(f"  전체 행 수:        {meta.num_rows}")
print(f"  Row Group 수:      {meta.num_row_groups}")
print(f"  컬럼 수:           {meta.num_columns}")

In [ ]:
# 스키마 확인
print("[ Schema ]")
print(pf.schema_arrow)

In [ ]:
# Row Group 분석
for rg_idx in range(meta.num_row_groups):
    rg = meta.row_group(rg_idx)
    print(f"\n[ Row Group {rg_idx} ]")
    print(f"  행 수:       {rg.num_rows}")
    print(f"  총 크기:     {rg.total_byte_size / 1024:.1f} KB")
    print(f"  컬럼 수:     {rg.num_columns}")
    
    print(f"\n  Column Chunks:")
    for col_idx in range(rg.num_columns):
        col = rg.column(col_idx)
        print(f"    [{col_idx}] {col.path_in_schema:20s} | "
              f"type={str(col.physical_type):10s} | "
              f"encodings={col.encodings} | "
              f"compressed={col.total_compressed_size} B | "
              f"uncompressed={col.total_uncompressed_size} B")

In [ ]:
# Column 통계 확인
rg = meta.row_group(0)
print("[ Column Statistics (Row Group 0) ]\n")
for col_idx in range(rg.num_columns):
    col = rg.column(col_idx)
    if col.statistics:
        stats = col.statistics
        print(f"  {col.path_in_schema}:")
        print(f"    num_values={stats.num_values}, null_count={stats.null_count}")
        if stats.has_min_max:
            print(f"    min={stats.min}, max={stats.max}")
        print()

## 파일 크기 및 레코드 수 통계

In [ ]:
print("[ 전체 Parquet 파일 통계 ]\n")

total_rows = 0
total_bytes = 0

for f in parquet_files:
    pf_tmp = pq.ParquetFile(f)
    rows = pf_tmp.metadata.num_rows
    size = os.path.getsize(f)
    total_rows += rows
    total_bytes += size
    rel = os.path.relpath(f, TABLE_PATH)
    print(f"  {rel}: {rows}행, {size / 1024:.1f} KB")

print(f"\n  합계: {total_rows}행, {len(parquet_files)}개 파일, {total_bytes / 1024:.1f} KB")
if len(parquet_files) > 0:
    print(f"  파일당 평균: {total_rows / len(parquet_files):.0f}행, {total_bytes / len(parquet_files) / 1024:.1f} KB")

## 관찰 포인트

### 1. Parquet = Columnar Format
- 스키마를 보면 각 컬럼이 독립적으로 저장됩니다
- `SELECT amount FROM ...` 같은 쿼리는 amount 컬럼만 읽으면 됩니다
- 압축 효율도 같은 타입의 데이터를 모아 저장하므로 높습니다

### 2. Column Statistics
- 각 Row Group의 각 컬럼에 `min`, `max`, `null_count` 통계가 저장됩니다
- 쿼리 엔진이 이 통계를 활용하여 불필요한 Row Group을 건너뛸 수 있습니다 (**Predicate Pushdown**)

### 3. Small File Problem ⚠️
- **파일 하나당 레코드가 매우 적습니다!**
- 파티션별로 1개의 작은 파일이 생성되었는데, 이것이 누적되면 수천~수만 개의 작은 파일이 됩니다
- 작은 파일이 많으면:
  - 메타데이터 오버헤드가 증가
  - 파일 열기/닫기 비용이 증가
  - 쿼리 계획 시간이 길어짐
- **이 문제는 `4_optimization`에서 Compaction을 통해 해결합니다**

In [ ]:
spark.stop()
print("Spark 세션 종료")